<a href="https://colab.research.google.com/github/nikhilkoditala/CSC-8980-NLP/blob/main/NLP_HW_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Name: Nikhil Koditala

# Panther ID: 002571023

In [1]:
import os
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
nltk.download('vader_lexicon')
from heapq import nlargest
import random, copy
import sklearn.metrics
import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

In [ ]:
# unzip data
!tar -xvf  '/content/review_polarity.tar.gz' -C '/content/'

In [4]:
positive_file_paths = os.listdir('/content/txt_sentoken/pos')
negative_file_paths = os.listdir('/content/txt_sentoken/neg') 

In [5]:
pos_text = []
for path in positive_file_paths:
  f = open(os.path.join('/content/txt_sentoken/pos',path),'r')
  pos_text.append(f.read())

In [6]:
neg_text = []
for path in negative_file_paths:
  f = open(os.path.join('/content/txt_sentoken/neg',path),'r')
  neg_text.append(f.read())

1 Using NLTK tokenize all documents, separated by polarity, ​remove stop words​, and list the top 20 most frequent tokens (and their counts) for the positive reviews, and the top 20 most frequent tokens (and their counts). What kind of things do you notice are different between the two sets? (30 points)

In [7]:
stop_words = list(stopwords.words('english')) 

In [8]:
#pos
pos_tokens = []
for pos_file in pos_text:
  pos_tokens.extend(nltk.word_tokenize(pos_file))

In [9]:
pos_tokens_after_stop_words = []

for token in pos_tokens:
  if(token not in stop_words):
    pos_tokens_after_stop_words.append(token)

In [10]:
pos_frequency_dict = {}

for token in pos_tokens_after_stop_words:
  if(token in pos_frequency_dict):
    pos_frequency_dict[token] += 1
  else:
    pos_frequency_dict[token] = 1

In [11]:
pos_ten_most_frequent_tokens = nlargest(20, pos_frequency_dict, key=pos_frequency_dict.get)

for token in pos_ten_most_frequent_tokens:
  print(token,' - ',pos_frequency_dict[token])

,  -  42448
.  -  33714
's  -  9473
``  -  8494
)  -  6039
(  -  6014
film  -  5186
one  -  2943
n't  -  2775
movie  -  2497
like  -  1713
?  -  1570
:  -  1502
story  -  1231
also  -  1200
good  -  1190
even  -  1175
time  -  1171
would  -  1079
character  -  1067


In [12]:
#neg
neg_tokens = []
for neg_file in neg_text:
  neg_tokens.extend(nltk.word_tokenize(neg_file))

In [13]:
neg_tokens_after_stop_words = []

for token in neg_tokens:
  if(token not in stop_words):
    neg_tokens_after_stop_words.append(token)

In [14]:
neg_frequency_dict = {}

for token in neg_tokens_after_stop_words:
  if(token in neg_frequency_dict):
    neg_frequency_dict[token] += 1
  else:
    neg_frequency_dict[token] = 1

In [15]:
neg_ten_most_frequent_tokens = nlargest(20, neg_frequency_dict, key=neg_frequency_dict.get)

for token in neg_ten_most_frequent_tokens:
  print(token,' - ',neg_frequency_dict[token])

,  -  35269
.  -  32162
``  -  9123
's  -  8655
)  -  5742
(  -  5650
film  -  4257
n't  -  3442
movie  -  3174
one  -  2637
?  -  2201
like  -  1832
:  -  1540
even  -  1381
would  -  1185
good  -  1126
time  -  1111
!  -  1056
get  -  1039
bad  -  1019


**What kind of things do you notice are different between the two sets?**

Tokens from positive documents have a lot of words with positive sentiments such as good. Whereas tokens from negative sentences contains words such as bad which are of negative sentiement.  


There are also few tokens which are frequently used in both positive and negative documents.

2 Using the code from previous lectures, build 3 polarity classifiers using the following parameters (20 points). Note: just train the models.

a) For training: use 50% of the positive dataset and 70% of the negative dataset. For your model use: NaiveBayes with the TF-IDF vectorizer.



In [16]:
def generate_splits(pos_percentage, neg_percentage):
  pos_text_r = copy.deepcopy(pos_text)
  random.seed(123)
  random.shuffle(pos_text_r)

  neg_text_r = copy.deepcopy(neg_text)
  random.shuffle(neg_text_r)

  pos_index = int((len(pos_text_r)/100)*pos_percentage)
  neg_index = int((len(neg_text_r)/100)*neg_percentage)

  return pos_text_r[:pos_index], pos_text_r[pos_index:], neg_text_r[:neg_index], neg_text_r[neg_index:]

In [17]:
pos_train_a, pos_test_a, neg_train_a, neg_test_a = generate_splits(50,70)

In [22]:
train_x_a = pos_train_a + neg_train_a
train_y_a = [1]*len(pos_train_a) + [0]*len(neg_train_a)

c = list(zip(train_x_a, train_y_a))
random.shuffle(c)
train_x_a, train_y_a = zip(*c)

In [27]:
model_a = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_a.fit(train_x_a, train_y_a)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

b) For training: use 50% of the negative dataset and 70% of the positive dataset.For your model use: NaiveBayes with the TF-IDF vectorizer.


In [28]:
pos_train_b, pos_test_b, neg_train_b, neg_test_b = generate_splits(70,50)

In [29]:
train_x_b = pos_train_b + neg_train_b
train_y_b = [1]*len(pos_train_b) + [0]*len(neg_train_b)

c = list(zip(train_x_b, train_y_b))
random.shuffle(c)
train_x_b, train_y_b = zip(*c)

In [30]:
model_b = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_b.fit(train_x_b, train_y_b)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

c) For training: use 25% of the negative dataset and 25% of the positive dataset. For your model use: SVM with the TF-IDF vectorizer.

In [31]:
pos_train_c, pos_test_c, neg_train_c, neg_test_c = generate_splits(25,25)

In [32]:
train_x_c = pos_train_c + neg_train_c
train_y_c = [1]*len(pos_train_c) + [0]*len(neg_train_c)

c = list(zip(train_x_c, train_y_c))
random.shuffle(c)
train_x_c, train_y_c = zip(*c)

In [33]:
model_c = make_pipeline(TfidfVectorizer(), SVC())
model_c.fit(train_x_c,train_y_c)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape=

3 Using the models from question 2, evaluate them on their individual rest of the dataset. This is, for a) 50% positive and 30% negative, for b) 50% negative and 30% positive, and for c) 75% negative and 75% positive. Calculate and show ONLY the following metrics for each model: Accuracy, Precision, Recall, Macro F1-score. (15 points).

In [38]:
y_test_model_a = [1]*len(pos_test_a) + [0]*len(neg_test_a)
y_pred_model_a = model_a.predict(pos_test_a + neg_test_a)
model_a_accuracy = sklearn.metrics.accuracy_score(y_pred_model_a,y_test_model_a)
model_a_recall = sklearn.metrics.recall_score(y_test_model_a,y_pred_model_a)
model_a_percision =  sklearn.metrics.precision_score(y_test_model_a,y_pred_model_a)
model_a_f1score =  sklearn.metrics.f1_score(y_test_model_a,y_pred_model_a,average='macro')

In [39]:
y_test_model_b = [1]*len(pos_test_b) + [0]*len(neg_test_b)
y_pred_model_b = model_b.predict(pos_test_b + neg_test_b)
model_b_accuracy = sklearn.metrics.accuracy_score(y_pred_model_b,y_test_model_b)
model_b_recall = sklearn.metrics.recall_score(y_test_model_b,y_pred_model_b)
model_b_percision =  sklearn.metrics.precision_score(y_test_model_b,y_pred_model_b)
model_b_f1score =  sklearn.metrics.f1_score(y_test_model_b,y_pred_model_b,average='macro')

In [40]:
y_test_model_c = [1]*len(pos_test_c) + [0]*len(neg_test_c)
y_pred_model_c = model_c.predict(pos_test_c + neg_test_c)
model_c_accuracy = sklearn.metrics.accuracy_score(y_pred_model_c,y_test_model_c)
model_c_recall = sklearn.metrics.recall_score(y_test_model_c,y_pred_model_c)
model_c_percision =  sklearn.metrics.precision_score(y_test_model_c,y_pred_model_c)
model_c_f1score =  sklearn.metrics.f1_score(y_test_model_c,y_pred_model_c,average='macro')

In [41]:
pd.DataFrame(np.array([['Model A',50,70,model_a_accuracy,model_a_recall,model_a_percision,model_a_f1score],
                             ['Model B',70,50,model_b_accuracy,model_b_recall,model_b_percision,model_b_f1score],
                             ['Model C',25,25,model_c_accuracy,model_c_recall,model_c_percision,model_c_f1score],
]),columns=['Model Name','Positive Percentage','Negative Percentage','Accuracy', 'Recall', 'Precision','F1-Score'])

,Model Name,Positive Percentage,Negative Percentage,Accuracy,Recall,Precision,F1-Score
0,Model A,50,70,0.37625,0.002,1.0,0.2749714401951329
1,Model B,70,50,0.375,1.0,0.375,0.2727272727272727
2,Model C,25,25,0.7526666666666667,0.7786666666666666,0.7401774397972116,0.7524993562314792


**4) Using the model performance metrics from question 3, answer the following questions. Please provide logical and intuitive rationale for your answers, simple answers like: because it has the best score, will not be sufficient. (40 points):**





**a) What is the best performing model?**


SVM with TFIDF is the best performing model

**b) Why do you think this is the best performing model?**

SVM performed best because the training set is balanced. For model 1 and model 2 the training set is highly imbalanced which might have affected model's performance. But for Model C the training set is balanced and it led to better performance.

**c) How does class imbalance play in determining polarity?**

By looking at the above metrics table, it clearly shows that model A and model B are affected by class imbalances. For model A, the percision is 1 and recall is 0.002. This clearly shows that the model is predicting 0 for all the values. This is because the training dataset for Model A is highly unbalanced with negative labels. 

Similarly, for model B, the recall is 1 and precision is 0.375. As model B is inbalanced with large number of positive the model is predicting 1 for all the values.

Due to the imbalance in the dataset, the models A and B have given importance to positive and negative labels respectively. Whereas Model C, being balanced dataset, performed better than the above two.


**d) Do you think either more data or a better model is a better approach for this kind of task?**

Depeding on the model, the performance of our classification task may improve. If we have large data, by using deep neural networks we might be able to build a better performing model.

But, having more data doesn't always mean that the performance of the model will improve as it might lead to overfitting of the model. So we have to choose the model and dataset optimally such that it doesn't lead to overfitting or underfitting.

**5) Using NLTK and VADER, calculate the sentiment score for all documents in the positive polarity. Calculate the polarity threshold needed (and reasonable) to have the majority of the document labels match. Do the same for the negative class. Provide the threshold needed, the reason why you think this threshold is reasonable, and the accuracy percentage (how many documents are correctly labeled using this threshold). (45 points):**

In [257]:
sia = SIA()

In [258]:
positive_sentiments = pd.DataFrame()

In [259]:
for path,doc in zip(positive_file_paths,pos_text):
  temp = sia.polarity_scores(doc)
  positive_sentiments = positive_sentiments.append([[path,temp['compound'],temp['neg'],temp['neu'],temp['pos']]])


In [260]:
positive_sentiments.columns = ['Filename','Compound','Negative','Neutral','Positive']

In [261]:
positive_sentiments

,Filename,Compound,Negative,Neutral,Positive
0,cv815_22456.txt,0.9931,0.104,0.736,0.160
0,cv274_25253.txt,0.9996,0.083,0.713,0.204
0,cv927_10681.txt,0.9210,0.069,0.852,0.079
0,cv442_13846.txt,-0.9993,0.199,0.684,0.117
0,cv551_10565.txt,0.9973,0.087,0.753,0.160
...,...,...,...,...,...
0,cv117_24295.txt,-0.9568,0.169,0.674,0.157
0,cv602_8300.txt,-0.9861,0.100,0.848,0.052
0,cv560_17175.txt,0.9960,0.107,0.759,0.134
0,cv891_6385.txt,-0.9181,0.120,0.771,0.109


In [304]:
thresholds = [-0.1*x for x in range(9,0,-1)] + [0.1*x for x in range(0,10)] # creating list of threshold values between 0 to 1 in intervals of 0.05

In [305]:
pos_accuracies = []
for threshold in thresholds:
  pos_accuracies.append(len(positive_sentiments.loc[positive_sentiments['Compound'] >= threshold])/len(positive_sentiments))

In [306]:
pos_threshold_accuracies = pd.DataFrame(pos_accuracies,thresholds)
pos_threshold_accuracies = pd.DataFrame({'Threshold':thresholds,'Accuracy':pos_accuracies})

In [307]:
pos_threshold_accuracies

,Threshold,Accuracy
0,-0.9,0.884
1,-0.8,0.867
2,-0.7,0.858
3,-0.6,0.851
4,-0.5,0.843
5,-0.4,0.840
6,-0.3,0.834
7,-0.2,0.831
8,-0.1,0.831
9,0.0,0.828


The above dataframe shows the threshold and accuracy of the positive dataset at that corresponding thresholds. The optimal threshold value can be 0.2 as there is no much increase in accuracy as we reduce the threshold down. 

In [308]:
negative_sentiments = pd.DataFrame()

In [309]:
for path,doc in zip(negative_file_paths,neg_text):
  temp = sia.polarity_scores(doc)
  negative_sentiments = negative_sentiments.append([[path,temp['compound'],temp['neg'],temp['neu'],temp['pos']]])

In [310]:
negative_sentiments.columns = ['Filename', 'Compound','Negative','Neutral','Positive']

In [311]:
negative_sentiments

,Filename,Compound,Negative,Neutral,Positive
0,cv561_9484.txt,-0.8243,0.109,0.798,0.093
0,cv135_12506.txt,0.9932,0.083,0.755,0.161
0,cv568_17065.txt,-0.5106,0.116,0.772,0.112
0,cv297_10104.txt,-0.6371,0.134,0.732,0.135
0,cv523_18285.txt,0.9387,0.067,0.833,0.100
...,...,...,...,...,...
0,cv323_29633.txt,-0.9766,0.146,0.729,0.125
0,cv574_23191.txt,0.9981,0.096,0.735,0.169
0,cv385_29621.txt,0.8535,0.108,0.758,0.134
0,cv936_17473.txt,0.9793,0.113,0.731,0.156


In [312]:
neg_accuracies = []
for threshold in thresholds:
  neg_accuracies.append(len(negative_sentiments.loc[negative_sentiments['Compound'] >= threshold])/len(negative_sentiments))

In [313]:
neg_threshold_accuracies = pd.DataFrame(neg_accuracies,thresholds)
neg_threshold_accuracies = pd.DataFrame({'Threshold':thresholds,'Accuracy':neg_accuracies})

In [314]:
neg_threshold_accuracies

,Threshold,Accuracy
0,-0.9,0.684
1,-0.8,0.642
2,-0.7,0.616
3,-0.6,0.597
4,-0.5,0.583
5,-0.4,0.579
6,-0.3,0.564
7,-0.2,0.561
8,-0.1,0.556
9,0.0,0.555


The above dataframe shows the threshold and accuracy of the negative dataset at that corresponding thresholds. The maximum threshold value to get atleast 50% of accuracy is at 0.6. The optimal threshold value can be -0.5 as we can see an elbow curve forming there.


**Bonus (40 points): Repeat questions 2,3 and 4 removing all stopwords. Answer the following questions: Did this change the results in any way? Why do you think so?**

Removing stopwords

In [198]:
pos_text_2 = pos_text
neg_text_2 = neg_text

In [199]:
pos_text_after_stop_words = []

for text in pos_text_2:
  temp = []
  tokens = nltk.word_tokenize(text)
  for token in tokens:
    if(token not in stop_words):
      temp.append(token)
  pos_text_after_stop_words.append(' '.join(temp))

In [200]:
neg_text_after_stop_words = []

for text in neg_text_2:
  temp = []
  tokens = nltk.word_tokenize(text)
  for token in tokens:
    if(token not in stop_words):
      temp.append(token)
  neg_text_after_stop_words.append(' '.join(temp))

In [201]:
def generate_splits_stopwords(pos_percentage, neg_percentage): # function used to split test and train data
  pos_text_r = copy.deepcopy(pos_text_after_stop_words)
  random.seed(123)
  random.shuffle(pos_text_r)

  neg_text_r = copy.deepcopy(neg_text_after_stop_words)
  random.shuffle(neg_text_r)

  pos_index = int((len(pos_text_r)/100)*pos_percentage)
  neg_index = int((len(neg_text_r)/100)*neg_percentage)

  return pos_text_r[:pos_index], pos_text_r[pos_index:], neg_text_r[:neg_index], neg_text_r[neg_index:]

Model Training

In [215]:
pos_train_a, pos_test_a, neg_train_a, neg_test_a = generate_splits_stopwords(50,70)

In [216]:
train_x_a = pos_train_a + neg_train_a
train_y_a = [1]*len(pos_train_a) + [0]*len(neg_train_a)

c = list(zip(train_x_a, train_y_a))
random.shuffle(c)
train_x_a, train_y_a = zip(*c)

In [217]:
model_a = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_a.fit(train_x_a, train_y_a)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [218]:
pos_train_b, pos_test_b, neg_train_b, neg_test_b = generate_splits_stopwords(70,50)

In [219]:
train_x_b = pos_train_b + neg_train_b
train_y_b = [1]*len(pos_train_b) + [0]*len(neg_train_b)

c = list(zip(train_x_b, train_y_b))
random.shuffle(c)
train_x_b, train_y_b = zip(*c)

In [220]:
model_b = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_b.fit(train_x_b, train_y_b)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [221]:
pos_train_c, pos_test_c, neg_train_c, neg_test_c = generate_splits_stopwords(25,25)

In [222]:
train_x_c = pos_train_c + neg_train_c
train_y_c = [1]*len(pos_train_c) + [0]*len(neg_train_c)

c = list(zip(train_x_c, train_y_c))
random.shuffle(c)
train_x_c, train_y_c = zip(*c)

In [223]:
model_c = make_pipeline(TfidfVectorizer(), SVC())
model_c.fit(train_x_c,train_y_c)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape=

Performance Metrics

In [224]:
y_test_model_a = [1]*len(pos_test_a) + [0]*len(neg_test_a)
y_pred_model_a = model_a.predict(pos_test_a + neg_test_a)
model_a_accuracy = sklearn.metrics.accuracy_score(y_pred_model_a,y_test_model_a)
model_a_recall = sklearn.metrics.recall_score(y_pred_model_a,y_test_model_a)
model_a_percision =  sklearn.metrics.precision_score(y_pred_model_a,y_test_model_a)
model_a_f1score =  sklearn.metrics.f1_score(y_pred_model_a,y_test_model_a,average='macro')

In [225]:
y_test_model_b = [1]*len(pos_test_b) + [0]*len(neg_test_b)
y_pred_model_b = model_b.predict(pos_test_b + neg_test_b)
model_b_accuracy = sklearn.metrics.accuracy_score(y_pred_model_b,y_test_model_b)
model_b_recall = sklearn.metrics.recall_score(y_pred_model_b,y_test_model_b)
model_b_percision =  sklearn.metrics.precision_score(y_pred_model_b,y_test_model_b)
model_b_f1score =  sklearn.metrics.f1_score(y_pred_model_b,y_test_model_b,average='macro')

In [226]:
y_test_model_c = [1]*len(pos_test_c) + [0]*len(neg_test_c)
y_pred_model_c = model_c.predict(pos_test_c + neg_test_c)
model_c_accuracy = sklearn.metrics.accuracy_score(y_pred_model_c,y_test_model_c)
model_c_recall = sklearn.metrics.recall_score(y_pred_model_c,y_test_model_c)
model_c_percision =  sklearn.metrics.precision_score(y_pred_model_c,y_test_model_c)
model_c_f1score =  sklearn.metrics.f1_score(y_pred_model_c,y_test_model_c,average='macro')

In [227]:
pd.DataFrame(np.array([['Model A',50,70,model_a_accuracy,model_a_recall,model_a_percision,model_a_f1score],
                             ['Model B',70,50,model_b_accuracy,model_b_recall,model_b_percision,model_b_f1score],
                             ['Model C',25,25,model_c_accuracy,model_c_recall,model_c_percision,model_c_f1score],
]),columns=['Model Name','Positive Percentage','Negative Percentage','Accuracy', 'Recall', 'Precision','F1-Score'])

,Model Name,Positive Percentage,Negative Percentage,Accuracy,Recall,Precision,F1-Score
0,Model A,50,70,0.38875,1.0,0.022,0.2970085124506046
1,Model B,70,50,0.38125,0.37735849056603776,1.0,0.283873592838736
2,Model C,25,25,0.764,0.7632978723404256,0.7653333333333333,0.7639995804436986


a) What is the best performing model?


Model C, which is SVM with TFIDF is the best performing model

b) Why do you think this is the best performing model?


SVM performed best because the training set is balanced. For model 1 and model 2 the training set is highly imbalanced which might have affected model's performance. But for Model C the training set is balanced and it led to better performance.

c) How does class imbalance play in determining polarity?

By looking at the above metrics table, it clearly shows that model A and model B are affected by class imbalances. For model A, the percision is 1 and recall is 0.022. This clearly shows that the model is predicting 0 for all the values. This is because the training dataset for Model A is highly unbalanced with negative labels.

Similarly, for model B, the recall is 1 and precision is 0.377. As model B is inbalanced with large number of positive the model is predicting 1 for all the values.

Due to the imbalance in the dataset, the models A and B have given importance to positive and negative labels respectively. Whereas Model C, being balanced dataset, performed better than the above two.

d) Do you think either more data or a better model is a better approach for this kind of task?

Depeding on the model, the performance of our classification task may improve. If we have large data, by using deep neural networks we might be able to build a better performing model.

But, having more data doesn't always mean that the performance of the model will improve as it might lead to overfitting of the model. So we have to choose the model and dataset optimally such that it doesn't lead to overfitting or underfitting.

 **Did this change the results in any way? Why do you think so?**

Yes, removing stopwords from the dataset has increased the performance of all the models. The accuracy, recall, precision and F1 Score have been improved for all the models because of removing stopwords.

Stop words are the words which are frequently used in the any text. For example, a, is, I, the. These type of words doesn't have much significance in determining the sentiment of the text when compared to other nouns, verbs and adverbs. So, by removing stop words the models have been able to give importance to specific words which provide the sentiment of overall text better.